# <center> <img src="../../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> Computer Systems Engineering  </center>
---
### <center> Big Data Processing </center>
---
#### <center> **Autumn 2025** </center>
---

**Lab 04**: Data Unions & Joins Pipeline

**Date**: September 23rd 2025

**Student Name**:

**Professor**: Pablo Camarillo Ramirez

In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Examples on SparkSQL") \
    .master("spark://b1ca502cde8a:7077") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/25 00:47:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [20]:
# Build schema
from auragutierrez.spark_utils import SparkUtils

schema_rentals = SparkUtils.generate_schema([("rental_id", "int"), ("rental_info", "string")])
df_rentals = spark.read.schema(schema_rentals).option("header", True).csv("/opt/spark/work-dir/data/car_service/rentals")

schema_agencies = SparkUtils.generate_schema([("agency_id", "int"), ("agency_info", "string")])
df_agencies = spark.read.schema(schema_agencies).option("header", True).csv("/opt/spark/work-dir/data/car_service/agencies")

schema_brands = SparkUtils.generate_schema([("brand_id", "int"), ("brand_info", "string")])
df_brands = spark.read.schema(schema_brands).option("header", True).csv("/opt/spark/work-dir/data/car_service/brands")

schema_cars = SparkUtils.generate_schema([("car_id", "int"), ("car_info", "string")])
df_cars = spark.read.schema(schema_cars).option("header", True).csv("/opt/spark/work-dir/data/car_service/cars")

schema_customers = SparkUtils.generate_schema([("customer_id", "int"), ("customer_info", "string")])
df_customers = spark.read.schema(schema_customers).option("header", True).csv("/opt/spark/work-dir/data/car_service/customers")

In [23]:
from pyspark.sql.functions import get_json_object, col

df_rentals_ids = (df_rentals.withColumn("car_id", get_json_object("rental_info", "$.car_id"))
                            .withColumn("customer_id", get_json_object("rental_info", "$.customer_id"))
                            .withColumn("agency_id", get_json_object("rental_info", "$.agency_id"))
                            .drop("rental_info")
                 )

+---------+------+-----------+---------+
|rental_id|car_id|customer_id|agency_id|
+---------+------+-----------+---------+
|    11891|    21|         71|        1|
|    11892|    11|         52|        2|
|    11893|    22|        116|        4|
|    11894|     5|        107|        1|
|    11895|     4|         53|        4|
+---------+------+-----------+---------+
only showing top 5 rows


In [25]:
df_rentals_join = (df_rentals_ids.join(df_cars, on="car_id", how="left")
                     .join(df_customers, on="customer_id", how="left")
                     .join(df_agencies, on="agency_id", how="left")
            )

+---------+-----------+------+---------+--------------------+--------------------+--------------------+
|agency_id|customer_id|car_id|rental_id|            car_info|       customer_info|         agency_info|
+---------+-----------+------+---------+--------------------+--------------------+--------------------+
|        1|         71|    21|    11891|{'car_name': 'Wal...|{'customer_name':...|{'agency_name': '...|
|        2|         52|    11|    11892|{'car_name': 'Gri...|{'customer_name':...|{'agency_name': '...|
|        4|        116|    22|    11893|{'car_name': 'Ste...|{'customer_name':...|{'agency_name': '...|
|        1|        107|     5|    11894|{'car_name': 'Cam...|{'customer_name':...|{'agency_name': '...|
|        4|         53|     4|    11895|{'car_name': 'Wag...|{'customer_name':...|{'agency_name': '...|
+---------+-----------+------+---------+--------------------+--------------------+--------------------+
only showing top 5 rows


In [26]:
import pyspark.sql.functions as F
df_rentals_final = (df_rentals_join.select("rental_id", 
                F.get_json_object(F.col("car_info"), "$.car_name").alias("car_name"),
                F.get_json_object(F.col("agency_info"), "$.agency_name").alias("agency_name"),
                F.get_json_object(F.col("customer_info"), "$.customer_name").alias("customer_name")))

In [27]:
df_rentals_final.show(n=5)

+---------+--------------------+-------------+---------------+
|rental_id|            car_name|  agency_name|  customer_name|
+---------+--------------------+-------------+---------------+
|    11891|Wallace-Carlson M...|  NYC Rentals| Margaret Jones|
|    11892|Grimes-Green Model 8|LA Car Rental|Albert Williams|
|    11893|Stewart-Allen Mod...|      SF Cars|  Caleb Fleming|
|    11894|  Campos PLC Model 4|  NYC Rentals|  Andrew Butler|
|    11895|  Wagner LLC Model 1|      SF Cars|  Kristin Potts|
+---------+--------------------+-------------+---------------+
only showing top 5 rows


In [5]:
sc.stop()